In [11]:
const fs = require('fs');
const parse = require("csv-parse/lib/sync");
const { AnomalyDetectorClient } = require('@azure/ai-anomaly-detector');
const { AzureKeyCredential } = require('@azure/core-auth');

In [12]:
const dotenv = require("dotenv");
dotenv.config();
// Authentication variables
// Add your Anomaly Detector subscription key and endpoint to your environment variables.
const key = process.env["API_KEY"] || "";
const endpoint = process.env["ENDPOINT"] || "";

In [13]:
//<vars>
// Spreadsheet with 2 columns and n rows.
let CSV_FILE = './request-data.csv';


In [14]:
// Points array for the request body
let points = [];
//</vars>

// <authentication>
let anomalyDetectorClient = new AnomalyDetectorClient(endpoint, new AzureKeyCredential(key));
// </authentication>


In [15]:
// <readFile>
function readFile() {
    let input = fs.readFileSync(CSV_FILE).toString();
    let parsed = parse(input, { skip_empty_lines: true });
    parsed.forEach(function (e) {
        points.push({ timestamp: new Date(e[0]), value: parseFloat(e[1]) });
    });
}
readFile()
// </readFile>


In [16]:

// <batchCall>
async function batchCall() {
    // Create request body for API call
    let body = { series: points, granularity: 'daily' }
    // Make the call to detect anomalies in whole series of points
    await anomalyDetectorClient.detectEntireSeries(body)
        .then((response) => {
            console.log("Batch (entire) anomaly detection):")
            for (let item = 0; item < response.isAnomaly.length; item++) {
                if (response.isAnomaly[item]) {
                    console.log("An anomaly was detected from the series, at row " + item)
                }
            }
        }).catch((error) => {
            console.log(error)
        })

}
batchCall()
// </batchCall>

Promise { <pending> }
Batch (entire) anomaly detection):
An anomaly was detected from the series, at row 3
An anomaly was detected from the series, at row 18
An anomaly was detected from the series, at row 21
An anomaly was detected from the series, at row 22
An anomaly was detected from the series, at row 23
An anomaly was detected from the series, at row 24
An anomaly was detected from the series, at row 25
An anomaly was detected from the series, at row 28
An anomaly was detected from the series, at row 29
An anomaly was detected from the series, at row 30
An anomaly was detected from the series, at row 31
An anomaly was detected from the series, at row 32
An anomaly was detected from the series, at row 35
An anomaly was detected from the series, at row 44


In [17]:
// <lastDetection>
async function lastDetection() {

    let body = { series: points, granularity: 'daily' }
    // Make the call to detect anomalies in the latest point of a series
    await anomalyDetectorClient.detectLastPoint(body)
        .then((response) => {
            console.log("Latest point anomaly detection:")
            if (response.isAnomaly) {
                console.log("The latest point, in row " + points.length + ", is detected as an anomaly.")
            } else {
                console.log("The latest point, in row " + points.length + ", is not detected as an anomaly.")
            }
        }).catch((error) => {
            console.log(error)
        })
}
lastDetection()
// </lastDetection>

Promise { <pending> }
Latest point anomaly detection:
The latest point, in row 47, is not detected as an anomaly.


In [18]:
// <changePointDetection>
async function changePointDetection() {

    let body = { series: points, granularity: 'daily' }
    // get change point detect results
    await anomalyDetectorClient.detectChangePoint(body)
        .then((response) => {
            if (
                response.isChangePoint.some(function (changePoint) {
                    return changePoint === true;
                })
            ) {
                console.log("Change points were detected from the series at index:");
                response.isChangePoint.forEach(function (changePoint, index) {
                    if (changePoint === true) {
                        console.log(index);
                    }
                });
            } else {
                console.log("There is no change point detected from the series.");
            }
        }).catch((error) => {
            console.log(error)
        })
}
// </changePointDetection>
changePointDetection();

Promise { <pending> }
Change points were detected from the series at index:
20
27
